<a href="https://colab.research.google.com/github/shaaagri/iat481-nlp-proj/blob/main/LLama2_vanilla_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction (by the project team)
In this preparation notebook, we will set up a suitable variant of Llama2 LLM to be able to do some raw queries. We then will try to integrate it into a langchain-based pipeline and test it by asking the model some simple questions (a "vanilla" chatbot, missing yet any customization). This will help us to get ready for our project's next iteration (in another notebok) in which we will take this pipeline one step further by implementing RAG (Retrieval Augmented Generation).

*Note*: This notebook is partially based on `Chatbot_LLama_2.ipynb` from Maryiam's tutorials and also heavily borrows from `Run LLama-2 on Google Colab` tutorial by Muhammad Moin ([link](https://github.com/MuhammadMoinFaisal/LargeLanguageModelsProjects/blob/main/Run%20Llama2%20Google%20Colab/Llama_2_updated.ipynb])).

# Workflow

1. Gathering the Dependencies
2.

# Gathering the Dependencies


`llama.cpp`'s objective is to run the LLaMA model with 4-bit integer quantization on MacBook. It is a plain C/C++ implementation optimized for Apple silicon and x86 architectures, supporting various integer quantization and BLAS libraries. Originally a web chat example, it now serves as a development playground for ggml library features.

`GGML`, a C library for machine learning, facilitates the distribution of large language models (LLMs). It utilizes quantization to enable efficient LLM execution on consumer hardware. GGML files contain binary-encoded data, including version number, hyperparameters, vocabulary, and weights. The vocabulary comprises tokens for language generation, while the weights determine the LLM's size. Quantization reduces precision to optimize resource usage.



## Quantized Models from the Hugging Face Community

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).

In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).


## Installing the Necessary Packages

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4